In [1]:
from pathlib import Path


from jmpfm.tasks.config import AdamWConfig
from jmpfm.tasks.finetune import (
    MatbenchConfig,
    MatbenchModel,
    PDBBindConfig,
    PDBBindModel,
    RMD17Config,
    RMD17Model,
)
from jmpfm.tasks.finetune.base import (
    PrimaryMetricConfig,
    FinetuneConfigBase,
    FinetuneModelBase,
)
from jmpfm.tasks.finetune import dataset_config as DC
from jmpfm.configs.finetune import jmp_l_ft_config_builder

CKPT_PATH = Path("/mnt/shared/checkpoints/fm_gnoc_large_2_epoch.ckpt")


def jmp_l_matbench_config(
    dataset: DC.MatbenchDataset,
    fold: DC.MatbenchFold,
    base_path: Path = Path("/mnt/shared/datasets/matbench/"),
):
    with jmp_l_ft_config_builder(MatbenchConfig, CKPT_PATH) as (builder, config):
        # Optimizer settings
        config.optimizer = AdamWConfig(
            lr=5.0e-6,
            amsgrad=False,
            betas=(0.9, 0.95),
            weight_decay=0.1,
        )

        # Set data config
        config.batch_size = 1

        # Set up dataset
        config.train_dataset = DC.matbench_config(dataset, base_path, "train", fold)
        config.val_dataset = DC.matbench_config(dataset, base_path, "val", fold)
        config.test_dataset = DC.matbench_config(dataset, base_path, "test", fold)

        # MatBench specific settings
        config.primary_metric = PrimaryMetricConfig(name="y_mae", mode="min")

        return builder(config), MatbenchModel


def jmp_l_rmd17_config(
    molecule: DC.RMD17Molecule,
    base_path: Path = Path("/mnt/shared/datasets/rmd17/"),
):
    with jmp_l_ft_config_builder(RMD17Config, CKPT_PATH) as (builder, config):
        # Optimizer settings
        config.optimizer = AdamWConfig(
            lr=5.0e-6,
            amsgrad=False,
            betas=(0.9, 0.95),
            weight_decay=0.1,
        )

        # Set data config
        config.batch_size = 4

        # Set up dataset
        config.train_dataset = DC.rmd17_config(molecule, base_path, "train")
        config.val_dataset = DC.rmd17_config(molecule, base_path, "val")
        config.test_dataset = DC.rmd17_config(molecule, base_path, "test")

        # RMD17 specific settings
        config.primary_metric = PrimaryMetricConfig(name="force_mae", mode="min")

        return builder(config), RMD17Model


def jmp_l_pdbbind_config():
    with jmp_l_ft_config_builder(PDBBindConfig, CKPT_PATH) as (builder, config):
        # Optimizer settings
        config.optimizer = AdamWConfig(
            lr=5.0e-6,
            amsgrad=False,
            betas=(0.9, 0.95),
            weight_decay=0.1,
        )

        # Set data config
        config.batch_size = 1

        # Set up dataset
        config.train_dataset = DC.pdbbind_config("train")
        config.val_dataset = DC.pdbbind_config("val")
        config.test_dataset = DC.pdbbind_config("test")

        # PDBBind specific settings
        config.pbdbind_task = "-logKd/Ki"
        config.metrics.report_rmse = True
        config.primary_metric = PrimaryMetricConfig(name="y_mae", mode="min")

        return builder(config), PDBBindModel


config, model_cls = jmp_l_pdbbind_config()
print(config)

configs: list[tuple[FinetuneConfigBase, type[FinetuneModelBase]]] = []
configs.append((config, model_cls))

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


id='gmfv7hg3' name=None project=None tags=[] notes=[] debug=False environment=EnvironmentConfig(cwd=None, python_executable=None, python_path=None, python_version=None, config=None, model=None, data=None, slurm=None, log_dir=None, seed=None, seed_workers=None, sweep_id=None, sweep_config=None) trainer=TrainerConfig(python_logging=PythonLogging(log_level=None, rich=True, rich_tracebacks=True, lovely_tensors=True, lovely_numpy=False), logging=LoggingConfig(enabled=True, log_lr=True, log_epoch=True, wandb=WandbLoggingConfig(enabled=True, log_model=False, watch=WandbWatchConfig(enabled=True, log=None, log_graph=True, log_freq=100)), csv=CSVLoggingConfig(enabled=True), tensorboard=TensorboardLoggingConfig(enabled=False)), optimizer=OptimizerConfig(grad_finite_checks=False, grad_none_checks=False, log_grad_norm=True, log_grad_norm_per_param=False, log_param_norm=False, log_param_norm_per_param=False, gradient_clipping=GradientClippingConfig(enabled=True, value=1.0, algorithm='value'), gradie

/workspaces/repositories/fm/src/ll/model/config.py:709: IdSeedWarning: BaseConfig._rng is None. The generated IDs will not be reproducible. To fix this, call BaseConfig.set_seed(...) before generating any IDs.
  warnings.warn(


In [2]:
from ll import Runner, Trainer

from jmpfm.utils.finetune_state_dict import (
    filter_state_dict,
    retreive_state_dict_for_finetuning,
)


def run(config: FinetuneConfigBase, model_cls: type[FinetuneModelBase]) -> None:
    if (ckpt_path := config.meta.get("ckpt_path")) is None:
        raise ValueError("No checkpoint path provided")

    model = model_cls(config)

    # Load the checkpoint
    state_dict = retreive_state_dict_for_finetuning(
        ckpt_path, load_emas=config.meta.get("ema_backbone", False)
    )
    embedding = filter_state_dict(state_dict, "embedding.atom_embedding.")
    backbone = filter_state_dict(state_dict, "backbone.")
    model.load_backbone_state_dict(backbone=backbone, embedding=embedding, strict=True)

    trainer = Trainer(config)
    trainer.fit(model)


runner = Runner(run)
runner.fast_dev_run(configs)

Fast dev run:   0%|          | 0/1 [00:00<?, ?it/s]

[22:33:36] CRITICAL Setting                                                                          ]8;id=338499;file:///workspaces/repositories/fm/src/ll/trainer/trainer.py\trainer.py]8;;\:]8;id=337339;file:///workspaces/repositories/fm/src/ll/trainer/trainer.py#159\159]8;;\
                    config.trainer.default_root_dir='/workspaces/repositories/fm/config/pretrain/lig               
                    htning_logs/cgmhfay1'.                                                                         

Seed set to 0


           CRITICAL Set global seed to 0.                                                                ]8;id=271493;file:///workspaces/repositories/fm/src/ll/util/seed.py\seed.py]8;;\:]8;id=536110;file:///workspaces/repositories/fm/src/ll/util/seed.py#11\11]8;;\

           CRITICAL Auto-wrapping run in Trainer context                                              ]8;id=499748;file:///workspaces/repositories/fm/src/ll/runner.py\runner.py]8;;\:]8;id=375441;file:///workspaces/repositories/fm/src/ll/runner.py#111\111]8;;\

...

`Trainer.fit` stopped: `max_steps=1` reached.


[22:33:50] CRITICAL Ran 1 finalizers for Trainer cleanup.                                            ]8;id=130488;file:///workspaces/repositories/fm/src/ll/trainer/trainer.py\trainer.py]8;;\:]8;id=665845;file:///workspaces/repositories/fm/src/ll/trainer/trainer.py#223\223]8;;\

Seed set to 0


           CRITICAL Reset global seed.                                                                   ]8;id=42450;file:///workspaces/repositories/fm/src/ll/util/seed.py\seed.py]8;;\:]8;id=271493;file:///workspaces/repositories/fm/src/ll/util/seed.py#17\17]8;;\

[None]